In [1]:
# Ensure compatible versions of libraries are installed
!pip install --upgrade transformers datasets scikit-learn pandas lightgbm numpy optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: fsspec
    Found existing i

In [ ]:
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import pandas as pd
import lightgbm as lgb

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import joblib

# Load the dictionary from the file
loaded_data = joblib.load('/content/gdrive/MyDrive/Colab Notebooks/Datasets/final/embeddings-and-labels_last-4-layers-avg_roberta-base.pkl')

# Access the data
train_embeddings = loaded_data['train_embeddings']
train_labels = loaded_data['train_labels']
test_embeddings = loaded_data['test_embeddings']
test_labels = loaded_data['test_labels']

print("Train and test embeddings and labels loaded using Joblib.")

Train and test embeddings and labels loaded using Joblib.


# Optuna automatic hyperparameter finetuning

In [ ]:
import lightgbm as lgb
import optuna
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from functools import partial # To pass fixed arguments to the objective function

# Assuming you have these from your embedding extraction process:
# train_embeddings: np.ndarray (shape: N_samples, N_features)
# train_labels: np.ndarray (shape: N_samples,)
# Example placeholders if you're running this standalone for testing:
# train_embeddings = np.random.rand(1000, 768)
# train_labels = np.random.randint(0, 2, 1000)


def objective(trial, X, y, n_splits=5, random_state=42):
    """
    Objective function for Optuna to optimize LightGBM hyperparameters.
    It performs K-Fold Cross-Validation for robust evaluation.
    """
    # 1. Define the hyperparameter search space using Optuna's trial object
    # These ranges are suggestions; you might need to adjust based on your data.
    param = {
        'objective': 'binary',             # Binary classification
        'metric': 'binary_logloss',        # Optimized for logloss, but evaluated with F1/ROC-AUC
        'boosting_type': 'gbdt',
        'n_jobs': -1,                      # Use all available CPU cores
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15), # Increased upper bound
        'num_leaves': trial.suggest_int('num_leaves', 20, 256), # Increased upper bound for more complexity
        'max_depth': trial.suggest_int('max_depth', 5, 20),      # Increased upper bound
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0), # Bagging fraction
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0), # Feature fraction
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 0.5), # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 0.5), # L2 regularization
        'random_state': random_state,
        'verbose': -1,                     # Suppress verbose output during training
        'n_estimators': 2000,              # A sufficiently large number, early stopping will manage
        # 'is_unbalance': True,              # Consider enabling if your classes are imbalanced
        # 'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0), # Use with is_unbalance=False if specific ratio known
    }

    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    f1_scores = []
    roc_auc_scores = []
    accuracy_scores = []

    # 2. Perform K-Fold Cross-Validation
    for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        model = lgb.LGBMClassifier(**param)

        # LightGBM's early stopping: Stop if validation metric doesn't improve for `patience` rounds
        callbacks = [lgb.early_stopping(stopping_rounds=100, verbose=False)] # Set verbose=True for detailed logs per fold

        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  eval_metric='logloss', # Or 'auc' if you prefer to stop based on ROC AUC
                  callbacks=callbacks)

        # Make predictions on the validation set
        y_pred_proba = model.predict_proba(X_val)[:, 1] # Probabilities for ROC AUC
        y_pred = model.predict(X_val)                   # Class labels for F1, Accuracy

        # Calculate metrics for the current fold
        f1 = f1_score(y_val, y_pred, average='macro') # Or 'weighted' if class imbalance is significant
        roc_auc = roc_auc_score(y_val, y_pred_proba)
        acc = accuracy_score(y_val, y_pred)

        f1_scores.append(f1)
        roc_auc_scores.append(roc_auc)
        accuracy_scores.append(acc)

        # Optuna pruning: A trial is pruned if its intermediate performance is not promising
        # This helps in early termination of unpromising trials.
        trial.report(np.mean(f1_scores), fold) # Report current mean F1 up to this fold
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # 3. Return the average metric across all folds
    # We'll optimize for average F1-score, as it's often a good balance for classification tasks.
    # You can change this to np.mean(roc_auc_scores) if ROC-AUC is your primary goal.
    return np.mean(f1_scores)


# --- Setup and Run Optuna Study ---

if __name__ == "__main__":


    # Create an Optuna study.
    # 'maximize' because we want to maximize F1-score.
    # You can specify a sampler (e.g., TPESampler is default and good).
    # You can also use a database for storage to resume studies (e.g., 'sqlite:///db.sqlite3').
    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))

    # Run the optimization.
    # n_trials: The number of different hyperparameter combinations Optuna will try.
    # Increase this number significantly for more thorough search (e.g., 200, 500, 1000).
    print(f"Starting Optuna optimization with {50} trials...")
    study.optimize(partial(objective, X=train_embeddings, y=train_labels, n_splits=5), n_trials=50, show_progress_bar=True)
    # The `partial` function is used to pass our X, y, and n_splits arguments to the objective function,
    # which Optuna expects to only take `trial` as its first argument.

    # 4. Print the best results
    print("\nOptimization finished.")
    print("Best trial:")
    print(f"  Value (Mean F1-score across folds): {study.best_value:.4f}")
    print("  Params: ")
    for key, value in study.best_params.items():
        print(f"    {key}: {value}")

[I 2025-06-15 20:56:01,353] A new study created in memory with name: no-name-99aa0399-529c-4f49-8b38-8afeb814c5c9


Starting Optuna optimization with 50 trials...


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ut

[I 2025-06-15 20:56:43,258] Trial 0 finished with value: 0.9954398359006464 and parameters: {'learning_rate': 0.062435616638630745, 'num_leaves': 245, 'max_depth': 16, 'min_child_samples': 64, 'subsample': 0.7468055921327309, 'colsample_bytree': 0.7467983561008608, 'reg_alpha': 0.02904180608409973, 'reg_lambda': 0.4330880728874676}. Best is trial 0 with value: 0.9954398359006464.


...

[I 2025-06-15 21:20:30,369] Trial 47 pruned. 


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ut

[I 2025-06-15 21:21:19,536] Trial 48 finished with value: 0.9955260797981966 and parameters: {'learning_rate': 0.030804149798786686, 'num_leaves': 138, 'max_depth': 12, 'min_child_samples': 92, 'subsample': 0.9775244688667297, 'colsample_bytree': 0.7524316263541948, 'reg_alpha': 0.4062716078108526, 'reg_lambda': 0.0950523600208984}. Best is trial 6 with value: 0.9955548263965825.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ut

[I 2025-06-15 21:22:19,494] Trial 49 pruned. 

Optimization finished.
Best trial:
  Value (Mean F1-score across folds): 0.9956
  Params: 
    learning_rate: 0.08653943910805914
    num_leaves: 63
    max_depth: 20
    min_child_samples: 80
    subsample: 0.9818496824692567
    colsample_bytree: 0.9684482051282947
    reg_alpha: 0.29894998940554257
    reg_lambda: 0.4609371175115584


# Training with best parameters from optuna

In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report

# --- 1. Define the Tuned Hyperparameters ---
# These are the best parameters you got from your Optuna run.
best_params = {
    'objective': 'binary',             # Binary classification
    'metric': 'binary_logloss',        # This is for internal evaluation during training
    'boosting_type': 'gbdt',
    'n_jobs': -1,                      # Use all available CPU cores
    'learning_rate': 0.08653943910805914,
    'num_leaves': 63,
    'max_depth': 20,
    'min_child_samples': 80,
    'subsample': 0.9818496824692567,
    'colsample_bytree': 0.9684482051282947,
    'reg_alpha': 0.29894998940554257,
    'reg_lambda': 0.4609371175115584,
    'random_state': 42,                # Ensure reproducibility
    'n_estimators': 2000,              # A high number, early stopping will prevent overfitting
    'verbose': -1                      # Suppress verbose output
    # If your dataset is imbalanced, consider adding:
    # 'is_unbalance': True,
    # or 'scale_pos_weight': your_calculated_weight,
}

# --- 3. Initialize and Train the Model ---
print("\nInitializing LightGBM model with best parameters...")
lgbm_model = lgb.LGBMClassifier(**best_params)


print("Training LightGBM model on the entire training dataset...")

from sklearn.model_selection import train_test_split

X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(
    train_embeddings, train_labels, test_size=0.1, random_state=42, stratify=train_labels
)

callbacks = [lgb.early_stopping(stopping_rounds=100, verbose=True)] # Set verbose=True to see early stopping in action

lgbm_model.fit(X_train_final, y_train_final,
              eval_set=[(X_val_final, y_val_final)],
              eval_metric='logloss', # Match the metric used in early stopping
              callbacks=callbacks)

print(f"Model trained. Best iteration found: {lgbm_model.best_iteration_}")


# --- 4. Evaluate the Model on the Test Set ---
print("\nEvaluating model on the test dataset...")
y_pred_proba_test = lgbm_model.predict_proba(test_embeddings)[:, 1]
y_pred_test = lgbm_model.predict(test_embeddings)

# Calculate metrics
f1_test = f1_score(test_labels, y_pred_test, average='macro')
roc_auc_test = roc_auc_score(test_labels, y_pred_proba_test)
accuracy_test = accuracy_score(test_labels, y_pred_test)

print(f"Test F1-score (macro): {f1_test:.4f}")
print(f"Test ROC AUC: {roc_auc_test:.4f}")
print(f"Test Accuracy: {accuracy_test:.4f}")

# Detailed classification report
print("\nClassification Report on Test Set:")
print(classification_report(test_labels, y_pred_test))


Initializing LightGBM model with best parameters...
Training LightGBM model on the entire training dataset...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[88]	valid_0's binary_logloss: 0.0248509
Model trained. Best iteration found: 88

Evaluating model on the test dataset...
Test F1-score (macro): 0.9713
Test ROC AUC: 0.9963
Test Accuracy: 0.9729

Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      5611
           1       0.97      0.96      0.96      3563

    accuracy                           0.97      9174
   macro avg       0.97      0.97      0.97      9174
weighted avg       0.97      0.97      0.97      9174



/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
joblib.dump(lgbm_model, '/content/gdrive/MyDrive/Colab Notebooks/lightgbm_models/lightgbm_roberta-base_last-4-layers-avg_early-stopping_hyperparameter-finetuned.pkl')

['/content/gdrive/MyDrive/Colab Notebooks/lightgbm_models/lightgbm_roberta-base_last-4-layers-avg_early-stopping_hyperparameter-finetuned.pkl']

In [ ]:
# First, ensure you have kaleido installed. Run this in a Colab cell:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
import plotly.io as pio
import optuna

# Set the renderer for Colab, as you already have.
# This ensures the plots are displayed correctly in the notebook output.
pio.renderers.default = "colab" # or "browser" if running locally

# --- Your existing Optuna study and plots generation ---
# Assuming 'study' object is already defined and optimization has been run.

print("\nPlotting optimization history...")
fig_hist = optuna.visualization.plot_optimization_history(study)
fig_hist.show()
# --- Save Optimization History Plot ---

fig_hist.write_html("optimization_history.html") # Saves as interactive HTML
fig_hist.write_image("optimization_history.png") # Saves as static image
print("Optimization history plot saved as optimization_history.png and .html")


print("\nPlotting parameter importances...")
fig_imp = optuna.visualization.plot_param_importances(study)
fig_imp.show()
# --- Save Parameter Importances Plot ---
fig_imp.write_html("parameter_importances.html") # Saves as interactive HTML
print("Parameter importances plot saved as parameter_importances.png and .html")


print("\nPlotting slice for parameter relationships...")
fig_slice = optuna.visualization.plot_slice(study)
fig_slice.show()
# --- Save Slice Plot ---
fig_slice.write_html("parameter_slice.html") # Saves as interactive HTML
print("Parameter slice plot saved as parameter_slice.png and .html")


Plotting optimization history...


NameError: name 'study' is not defined

# Obsolete

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Assuming train_embeddings, train_labels are your data
X_train_lgb, X_val_lgb, y_train_lgb, y_val_lgb = train_test_split(
    train_embeddings, train_labels, test_size=0.1, random_state=42, stratify=train_labels
)

lgb_clf = lgb.LGBMClassifier(objective='binary', metric='binary_logloss',
                             n_estimators=1000, # Start with a large number
                             learning_rate=0.05,
                             num_leaves=31,
                             max_depth=12,
                             random_state=42,
                             min_child_samples=100,
                             n_jobs=-1, # Use all available cores
                             colsample_bytree=0.8,
                             subsample=0.8,
                             reg_alpha=0.1,
                             reg_lambda=0.1)

lgb_clf.fit(X_train_lgb, y_train_lgb,
            eval_set=[(X_val_lgb, y_val_lgb)],
            eval_metric='binary_logloss',
            callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=True)])

[LightGBM] [Info] Number of positive: 12825, number of negative: 20200
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.218904 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 33025, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.388342 -> initscore=-0.454286
[LightGBM] [Info] Start training from score -0.454286
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,12
,learning_rate,0.05
,n_estimators,1000
,subsample_for_bin,200000
,objective,'binary'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,100


In [ ]:
# --- 7. Evaluate LightGBM Model ---
print("\nEvaluating LightGBM model on test embeddings...")

# Make predictions
lgb_predictions_proba = lgb_clf.predict_proba(test_embeddings)[:, 1] # Probability of positive class
lgb_predictions = (lgb_predictions_proba > 0.5).astype(int) # Convert probabilities to binary predictions

# Calculate metrics
lgb_accuracy = accuracy_score(test_labels, lgb_predictions)
lgb_f1 = f1_score(test_labels, lgb_predictions)
lgb_precision = precision_score(test_labels, lgb_predictions)
lgb_recall = recall_score(test_labels, lgb_predictions)

print(f"LightGBM Accuracy: {lgb_accuracy:.4f}")
print(f"LightGBM F1 Score: {lgb_f1:.4f}")
print(f"LightGBM Precision: {lgb_precision:.4f}")
print(f"LightGBM Recall: {lgb_recall:.4f}")


Evaluating LightGBM model on test embeddings...
LightGBM Accuracy: 0.9763
LightGBM F1 Score: 0.9694
LightGBM Precision: 0.9753
LightGBM Recall: 0.9635


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# --- 6. Train LightGBM Model ---
print("\nTraining LightGBM model...")

# Initialize LightGBM classifier
# You can tune these parameters further
lgb_clf = lgb.LGBMClassifier(objective='binary',
                             metric='binary_logloss',
                             n_estimators=1000, # Number of boosting rounds
                             learning_rate=0.05,
                             num_leaves=31,
                             max_depth=12,
                             random_state=42,
                             min_child_samples=100,
                             n_jobs=-1, # Use all available cores
                             colsample_bytree=0.8,
                             subsample=0.8,
                             reg_alpha=0.1,
                             reg_lambda=0.1)

# Train the model
lgb_clf.fit(train_embeddings, train_labels)

print("LightGBM training complete.")


Training LightGBM model...
[LightGBM] [Info] Number of positive: 14250, number of negative: 22445
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.310752 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 36695, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.388336 -> initscore=-0.454311
[LightGBM] [Info] Start training from score -0.454311
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

Exception ignored on calling ctypes callback function: <function _log_callback at 0x7a67503afce0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

Exception ignored on calling ctypes callback function: <function _log_callback at 0x7a67503afce0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po